In [1]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# example of random oversampling to balance the class distribution
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot
from tensorflow import keras


import tensorflow_addons as tfa
import tensorflow as tf

from scipy import stats
from imblearn.over_sampling import RandomOverSampler, SMOTE


2023-03-10 10:38:15.188711: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 10:38:15.655568: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-10 10:38:16.596271: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/tf/lib/
2023-03-10 10:38:16.598656: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7';

In [2]:
# configs
EPOCHS = 5
BATCH_SIZE = 32

TIME_SERIES_SIZE = 4   # Determines the window size. Ex (4,9)
TIME_STEP_SHIFT  = 1   # Determines specifies the number of steps to move the window forward at each iteration.

baseFolder = "../data_2019_processed/"

# selected features
inputFeatures = ["activity","location","day_of_week",
                 "light","phone_lock","proximity",
                 "sound","time_to_next_alarm", "minutes_day"]
outputClasses = ["awake","asleep"]
#outputClasses = ["class"]

In [3]:
# y_test     = Array with real values
# yhat_probs = Array with predicted values
def printMetrics(y_test,yhat_probs):
    # predict crisp classes for test set deprecated
    #yhat_classes = model.predict_classes(X_test, verbose=0)
    #yhat_classes = np.argmax(yhat_probs,axis=1)
    yhat_classes = yhat_probs.round()
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(y_test, yhat_classes)
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, yhat_classes)
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, yhat_classes)
    print('F1 score: %f' % f1)
    # kappa
    kappa = cohen_kappa_score(y_test, yhat_classes)
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    auc = roc_auc_score(y_test, yhat_probs)
    print('ROC AUC: %f' % auc)
    # confusion matrix
    print("\Confusion Matrix")
    matrix = confusion_matrix(y_test, yhat_classes)
    print(matrix)
    
    results = dict()
    results['accuracy'] = accuracy
    results['precision'] = precision
    results['recall'] = recall
    results['f1_score'] = f1
    results['cohen_kappa_score'] = kappa
    results['roc_auc_score'] = auc
    return results

def showGlobalMetrics(metrics):
    accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score = 0,0,0,0,0,0
    for metric in metrics:
        accuracy = accuracy + metric['accuracy']
        precision = precision + metric['precision']
        recall = recall + metric['recall']
        f1_score = f1_score + metric['f1_score']
        cohen_kappa_score = cohen_kappa_score + metric['cohen_kappa_score']
        roc_auc_score = roc_auc_score + metric['roc_auc_score']
        
    # mean
    size = len(metrics)
    print(size)
    accuracy = accuracy / size
    precision = precision / size
    recall = recall / size
    f1_score = f1_score / size
    cohen_kappa_score = cohen_kappa_score / size
    roc_auc_score = roc_auc_score / size
    
    #show:\
    print("accuracy: ",accuracy)
    print("precision: ",precision)
    print("recall: ",recall)
    print("f1_score: ",f1_score)
    print("cohen_kappa_score: ",cohen_kappa_score)
    print("roc_auc_score: ",roc_auc_score)
    
def transform_data_type(dataframe):
    
    # transform inputs
    for column in inputFeatures:
        if(dataframe[column].dtypes in ()):
            dataframe[column] = dataframe[column].astype('float32')
    
    # transform outputs
    #for column in outputClasses:
    #    if(dataframe[column].dtypes != object):
    #        dataframe[column] = dataframe[column].astype('float32')
    
    return dataframe

# one-hot encoding function
def transform_output_nominal_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data['awake']
    dataset['asleep'] = one_hot_encoded_data['asleep']
    
    return dataset

# one-hot encoding function
def transform_output_numerical_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data[0]
    dataset['asleep'] = one_hot_encoded_data[1]
    
    return dataset


def create_dataset_time_series_with_one_output(X, y, window_time_steps=1, shift_step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - window_time_steps, shift_step):
        v = X.iloc[i:(i + window_time_steps)].values
        labels = y.iloc[i: i + window_time_steps]
        Xs.append(v)        
        ys.append(stats.mode(labels)[0][0])
        
    if len(y.columns) == 1:
        return np.array(Xs), np.array(ys).reshape(-1, 1)
    else:
        return np.array(Xs), np.array(ys).reshape(-1, len(y.columns))

In [4]:
X_train = pd.read_csv(baseFolder+"train/allData-classification-numeric-normalized.csv")
X_test  = pd.read_csv(baseFolder+"test/allData-classification-numeric-normalized.csv")
#X_train = pd.read_csv(baseFolder+"train/allData-classification-numeric-normalized_balanced_undersample.csv")
#X_train  = pd.read_csv(baseFolder+"test/allData-classification-numeric-normalized_balanced_oversample.csv")

#AA = pd.read_csv(baseFolder+"allData-classification-numeric-normalized.csv")
#X_train, X_test = train_test_split(AA,test_size=0.25)

In [5]:
print(X_train.info())
X_train

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 407451 entries, 0 to 407450
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            407451 non-null  float64
 1   location            407451 non-null  float64
 2   timestamp           407451 non-null  float64
 3   time_to_next_alarm  407451 non-null  float64
 4   sound               407451 non-null  float64
 5   proximity           407451 non-null  float64
 6   phone_lock          407451 non-null  float64
 7   light               407451 non-null  float64
 8   day_of_week         407451 non-null  float64
 9   minutes_day         407451 non-null  float64
 10  timestamp_text      407451 non-null  object 
 11  class               407451 non-null  object 
dtypes: float64(10), object(2)
memory usage: 37.3+ MB
None


,activity,location,timestamp,time_to_next_alarm,sound,proximity,phone_lock,light,day_of_week,minutes_day,timestamp_text,class
0,0.00,0.0,0.000869,0.982143,0.000000,1.0,0.0,0.000175,0.0,0.597637,2018-05-15 14:20:45,awake
1,0.00,0.0,0.000869,0.982143,0.000000,1.0,0.0,0.000175,0.0,0.597637,2018-05-15 14:20:45,awake
2,0.25,0.5,0.000869,0.982044,0.604408,1.0,0.0,0.000165,0.0,0.598332,2018-05-15 14:21:15,awake
3,0.25,0.5,0.000869,0.982044,0.604408,1.0,0.0,0.001449,0.0,0.598332,2018-05-15 14:21:45,awake
4,0.25,0.5,0.000869,0.981944,0.601849,1.0,0.0,0.000198,0.0,0.599027,2018-05-15 14:22:15,awake
...,...,...,...,...,...,...,...,...,...,...,...,...
407446,0.25,1.0,0.006924,0.992956,0.644370,1.0,1.0,0.000000,0.0,0.549687,2018-06-12 13:11:39,awake
407447,0.25,1.0,0.006924,0.992956,0.644370,1.0,1.0,0.000000,0.0,0.550382,2018-06-12 13:12:09,awake
407448,0.25,1.0,0.006924,0.992857,0.624127,1.0,1.0,0.000538,0.0,0.551077,2018-06-12 13:13:37,awake
407449,0.00,1.0,0.006924,0.992758,0.540295,0.0,0.0,0.000000,0.0,0.551772,2018-06-12 13:14:07,awake


In [6]:
# transforms the input data to float32
X_test = transform_data_type(X_test)

# transforms the input data to float32
X_train = transform_data_type(X_train)


# balance
print(Counter(X_train['class']))
smote = SMOTE(random_state = 32)
X_train, y_train = smote.fit_resample(X_train[inputFeatures], X_train['class'])
X_train['class'] = y_train
print(Counter(X_train['class']))


# transform output to one_hot_encoding for the testing dataset
X_test = transform_output_nominal_class_into_one_hot_encoding(X_test)

# transform output to one_hot_encoding for the testing dataset
X_train = transform_output_nominal_class_into_one_hot_encoding(X_train)


Counter({'awake': 333593, 'asleep': 73858})
Counter({'awake': 333593, 'asleep': 333593})


In [7]:
# selects the data to train and test
X_train_data = pd.DataFrame(data=X_train,columns=inputFeatures)
y_train_data = pd.DataFrame(data=X_train,columns=outputClasses)
# selec test dataset (fixed to all)
X_test_data = pd.DataFrame(data=X_test,columns=inputFeatures)
y_test_data = pd.DataFrame(data=X_test,columns=outputClasses)

X_train_data, y_train_data = create_dataset_time_series_with_one_output(   #timestamp
    X_train_data, 
    y_train_data, 
    TIME_SERIES_SIZE, 
    TIME_STEP_SHIFT
)

X_test_data, y_test_data = create_dataset_time_series_with_one_output(    #timestamp
    X_test_data, 
    y_test_data, 
    TIME_SERIES_SIZE, 
    TIME_STEP_SHIFT
)


print("shape: ",X_train_data.shape, y_train_data.shape)
print("Size: ",X_test_data.shape,y_test_data.shape)       

shape:  (667182, 4, 9) (667182, 2)
Size:  (134884, 4, 9) (134884, 2)


In [8]:
# transtorm data to tensor slices
test_dataset_series = tf.data.Dataset.from_tensor_slices((X_test_data, y_test_data))
train_dataset_series = tf.data.Dataset.from_tensor_slices((X_train_data, y_train_data))

2023-03-10 10:42:04.673657: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-10 10:42:04.726181: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-10 10:42:04.726512: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-10 10:42:04.727681: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [9]:
train_dataset_series


<TensorSliceDataset element_spec=(TensorSpec(shape=(4, 9), dtype=tf.float64, name=None), TensorSpec(shape=(2,), dtype=tf.uint8, name=None))>

In [10]:
#client_test_dataset.window(size=4, shift=1, stride=1, drop_remainder=True)
#test_dataset_series.batch(BATCH_SIZE) # usado no federated learning

In [11]:
# batch data size
#test_dataset_series1 = test_dataset_series.batch(1)
#train_dataset_series1 = train_dataset_series.batch(1)

test_dataset_series1 = test_dataset_series.batch(BATCH_SIZE)
train_dataset_series1 = train_dataset_series.batch(BATCH_SIZE)

train_dataset_series1

<BatchDataset element_spec=(TensorSpec(shape=(None, 4, 9), dtype=tf.float64, name=None), TensorSpec(shape=(None, 2), dtype=tf.uint8, name=None))>

In [12]:
print("input_shape=[", X_train_data.shape[1], X_train_data.shape[2],"]")
print("output shape:",len(outputClasses))

verbose, epochs, batch_size = 1, 2, BATCH_SIZE
#verbose, epochs, batch_size = 1, 3, 16
model = keras.Sequential()
model.add(LSTM(128,activation="tanh", 
          input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
#model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(len(outputClasses), activation='softmax'))#softmax,sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy     
# fit network
#model.fit(train_dataset_series1, epochs=epochs, verbose=verbose) #, batch_size=batch_size, validation_split=0.1
model.fit(X_train_data, y_train_data, epochs=epochs, verbose=verbose, batch_size=batch_size) #, batch_size=batch_size, validation_split=0.1
# evaluate model
#accuracy = model.evaluate(X_test_data, y_test_data) # , batch_size=batch_size, verbose=0
#accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
# predict
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

input_shape=[ 4 9 ]
output shape: 2


2023-03-10 10:42:14.828436: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 96074208 exceeds 10% of free system memory.
2023-03-10 10:42:15.104326: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 96074208 exceeds 10% of free system memory.


Epoch 1/2


2023-03-10 10:42:21.222387: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201
2023-03-10 10:42:27.632326: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


20850/20850 [==============================] - 170s 8ms/step - loss: 0.2706 - categorical_accuracy: 0.8853
Epoch 2/2
4216/4216 [==============================] - 11s 3ms/step

awake
Accuracy: 0.829268
Precision: 0.857695
Recall: 0.947089
F1 score: 0.900178
Cohens kappa: 0.320547
ROC AUC: 0.772688
\Confusion Matrix
[[  8019  17228]
 [  5801 103836]]

asleep
Accuracy: 0.830276
Precision: 0.576845
Recall: 0.318663
F1 score: 0.410536
Cohens kappa: 0.320896
ROC AUC: 0.772838
\Confusion Matrix
[[104019   5848]
 [ 17045   7972]]

Global
2
accuracy:  0.8297722487470716
precision:  0.7172701276850326
recall:  0.6328761695565445
f1_score:  0.6553572484408251
cohen_kappa_score:  0.32072175262344094
roc_auc_score:  0.7727630140072326


In [13]:
print("input_shape=[", X_train_data.shape[1], X_train_data.shape[2],"]")
print("output shape:",len(outputClasses))

verbose, epochs, batch_size = 1, 7, BATCH_SIZE
#verbose, epochs, batch_size = 1, 3, 16
model = keras.Sequential()
model.add(LSTM(128,activation="tanh", 
          input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
#model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(len(outputClasses), activation='softmax'))#softmax,sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy     
# fit network
#model.fit(train_dataset_series1, epochs=epochs, verbose=verbose) #, batch_size=batch_size, validation_split=0.1
model.fit(X_train_data, y_train_data, epochs=epochs, verbose=verbose, batch_size=batch_size) #, batch_size=batch_size, validation_split=0.1
# evaluate model
#accuracy = model.evaluate(X_test_data, y_test_data) # , batch_size=batch_size, verbose=0
#accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
# predict
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

input_shape=[ 4 9 ]
output shape: 2
Epoch 1/7
20850/20850 [==============================] - 194s 9ms/step - loss: 0.2727 - categorical_accuracy: 0.8850
Epoch 2/7
20850/20850 [==============================] - 168s 8ms/step - loss: 0.2233 - categorical_accuracy: 0.9047
Epoch 3/7
20850/20850 [==============================] - 162s 8ms/step - loss: 0.2072 - categorical_accuracy: 0.9092
Epoch 4/7
20850/20850 [==============================] - 169s 8ms/step - loss: 0.1952 - categorical_accuracy: 0.9145
Epoch 5/7
20850/20850 [==============================] - 168s 8ms/step - loss: 0.1853 - categorical_accuracy: 0.9190
Epoch 6/7
20850/20850 [==============================] - 172s 8ms/step - loss: 0.1755 - categorical_accuracy: 0.9238
Epoch 7/7
4216/4216 [==============================] - 10s 2ms/step

awake
Accuracy: 0.822358
Precision: 0.857711
Recall: 0.936874
F1 score: 0.895546
Cohens kappa: 0.309716
ROC AUC: 0.760121
\Confusion Matrix
[[  8207  17040]
 [  6921 102716]]

asleep
Accuracy: 

In [14]:
print("input_shape=[", X_train_data.shape[1], X_train_data.shape[2],"]")
print("output shape:",len(outputClasses))

verbose, epochs, batch_size = 1, 15, BATCH_SIZE
#verbose, epochs, batch_size = 1, 3, 16
model = keras.Sequential()
model.add(LSTM(128,activation="tanh", 
          input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
#model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(len(outputClasses), activation='softmax'))#softmax,sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy     
# fit network
#model.fit(train_dataset_series1, epochs=epochs, verbose=verbose) #, batch_size=batch_size, validation_split=0.1
model.fit(X_train_data, y_train_data, epochs=epochs, verbose=verbose, batch_size=batch_size) #, batch_size=batch_size, validation_split=0.1
# evaluate model
#accuracy = model.evaluate(X_test_data, y_test_data) # , batch_size=batch_size, verbose=0
#accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
# predict
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

input_shape=[ 4 9 ]
output shape: 2
Epoch 1/15
20850/20850 [==============================] - 178s 9ms/step - loss: 0.2713 - categorical_accuracy: 0.8851
Epoch 2/15
20850/20850 [==============================] - 176s 8ms/step - loss: 0.2226 - categorical_accuracy: 0.9047
Epoch 3/15
20850/20850 [==============================] - 174s 8ms/step - loss: 0.2062 - categorical_accuracy: 0.9108
Epoch 4/15
20850/20850 [==============================] - 176s 8ms/step - loss: 0.1949 - categorical_accuracy: 0.9148
Epoch 5/15
20850/20850 [==============================] - 176s 8ms/step - loss: 0.1860 - categorical_accuracy: 0.9185
Epoch 6/15
20850/20850 [==============================] - 176s 8ms/step - loss: 0.1786 - categorical_accuracy: 0.9218
Epoch 7/15
20850/20850 [==============================] - 179s 9ms/step - loss: 0.1715 - categorical_accuracy: 0.9251
Epoch 8/15
20850/20850 [==============================] - 174s 8ms/step - loss: 0.1656 - categorical_accuracy: 0.9282
Epoch 9/15
20850/208

In [18]:
print("input_shape=[", X_train_data.shape[1], X_train_data.shape[2],"]")
print("output shape:",len(outputClasses))

verbose, epochs, batch_size = 1, 100, BATCH_SIZE
#verbose, epochs, batch_size = 1, 3, 16
model = keras.Sequential()
model.add(LSTM(128,activation="tanh", 
          input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
#model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(len(outputClasses), activation='softmax'))#softmax,sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy     
# fit network
model.fit(train_dataset_series1, epochs=epochs, verbose=verbose, batch_size=batch_size) #, batch_size=batch_size, validation_split=0.1
# evaluate model
#accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
# predict
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

input_shape=[ 4 9 ]
output shape: 2
Epoch 1/100
6896/6896 [==============================] - 53s 7ms/step - loss: 0.2948 - categorical_accuracy: 0.8796
Epoch 2/100
6896/6896 [==============================] - 50s 7ms/step - loss: 0.2989 - categorical_accuracy: 0.8755
Epoch 3/100
6896/6896 [==============================] - 49s 7ms/step - loss: 0.3014 - categorical_accuracy: 0.8727
Epoch 4/100
6896/6896 [==============================] - 59s 9ms/step - loss: 0.3000 - categorical_accuracy: 0.8781
Epoch 5/100
6896/6896 [==============================] - 63s 9ms/step - loss: 0.2947 - categorical_accuracy: 0.8804
Epoch 6/100
6896/6896 [==============================] - 64s 9ms/step - loss: 0.2932 - categorical_accuracy: 0.8833
Epoch 7/100
6896/6896 [==============================] - 63s 9ms/step - loss: 0.2925 - categorical_accuracy: 0.8837
Epoch 8/100
6896/6896 [==============================] - 63s 9ms/step - loss: 0.2858 - categorical_accuracy: 0.8883
Epoch 9/100
6896/6896 [=============

In [20]:
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

4259/4259 [==============================] - 11s 2ms/step

awake
Accuracy: 0.272685
Precision: 0.979019
Recall: 0.102880
F1 score: 0.186194
Cohens kappa: 0.038272
ROC AUC: 0.737497
\Confusion Matrix
[[25823   243]
 [98877 11339]]

asleep
Accuracy: 0.270953
Precision: 0.205180
Recall: 0.990630
F1 score: 0.339950
Cohens kappa: 0.037826
ROC AUC: 0.706877
\Confusion Matrix
[[11340 99114]
 [  242 25586]]

Global
2
accuracy:  0.2718187288123156
precision:  0.5920998003566356
recall:  0.5467550616729189
f1_score:  0.263071788036283
cohen_kappa_score:  0.03804903634117718
roc_auc_score:  0.7221871440907924
